In [0]:
%pip install databricks-vectorsearch
dbutils.library.restartPython()

In [0]:
import time
from databricks.sdk import WorkspaceClient
from databricks.vector_search.client import VectorSearchClient

# Initialize Databricks workspace client
w = WorkspaceClient()

def create_and_wait_for_endpoint(endpoint_name: str, poll_interval: int = 10):
    """
    Creates a vector search endpoint and waits until its status is READY.
    
    :param endpoint_name: Name of the endpoint to create.
    :param endpoint_config: Configuration dictionary for the endpoint.
    :param poll_interval: Time interval (in seconds) to check endpoint status.
    """
    # Create endpoint
    
    client = VectorSearchClient(disable_notice=True)
    try:
        is_present = [1 if i['name'] == endpoint_name else 0 for i in client.list_endpoints()['endpoints']][0]
        if(is_present == 1):
           print(f"Endpoint {endpoint_name} already exists. Skipping creation...")
           return
    except Exception as e:
        print("Endpoint does not exist. Creating...")
    response = client.create_endpoint(name=endpoint_name, endpoint_type="STANDARD")
  
    
    # Wait until the endpoint is fully provisioned
    endpoint_ready = False
    retry_interval = 15  # Wait time in seconds between retries
    
    while not endpoint_ready:
        try:
            # Check if the endpoint is available
            ep = client.get_endpoint(endpoint_name)
            if ep.get('endpoint_status', {}).get('state', '') != 'ONLINE':
                print(f"Endpoint is almost ready. Sleep for {retry_interval * 2} seconds...")
                time.sleep(retry_interval * 2)
            print("Endpoint is ready...")
            endpoint_ready = True
        except Exception as e:
             print(f"Endpoint not ready, retrying in {retry_interval} seconds...")
             time.sleep(retry_interval)

# Example usage
endpoint_name = "databricks_llm_geekcoders"

create_and_wait_for_endpoint(endpoint_name)
